# Jupyter notebook based on ImageD11 to process 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 25/07/2024

This notebook will help you to extract the locations of diffraction peaks on your detector images.

It will also merge together your 2D spots (on a stack of detector images with different omega angles).

We merge across omega because we often see the same spot twice on multiple detector images.

The results are saved to the PROCESSED_DATA folder of the experiment, inside the sample and dataset folders that you select within this notebook

## NOTE: These notebooks are under active development
They require the latest version of ImageD11 from Git to run.

If you don't have this set up yet, you can run the below cell.

It will automatically download and install ImageD11 to your home directory

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import glob, pprint

import ImageD11.sinograms.dataset
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties

import numpy as np
import fabio
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage import filters, measure, morphology
import ipywidgets as widgets
import h5py
from IPython.display import display
%matplotlib widget

from ImageD11.nbGui import nb_utils as utils
from ImageD11.nbGui import segmenter_gui

from frelon_peaksearch import worker, process

from ImageD11.blobcorrector import correct_cf_with_spline


# Experts : update these files for your detector if you need to

splinefile = '/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/frelon36.spline'
bgfile = "/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/FeAu_0p5_tR/tdxrd_all/ff_bkg.edf"
maskfile = '/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/mask.edf'

detector = "frelon3"
omegamotor = "diffrz"
dtymotor = "diffty"

#Define the initial parameters
options = {
    "bgfile":bgfile,
    "maskfile":maskfile,
    "threshold":50,
    "smoothsigma":1.0,
    "bgc":0.9,
    "minpx":3,
    "m_offset_thresh":80,
    "m_ratio_thresh":135,
}

In [ ]:
# Set up the file paths. Edit this if you are not at ESRF or not using the latest data policy.
dataroot, analysisroot = segmenter_gui.guess_ESRF_paths() 

if len(dataroot)==0:
    print("Please fix in the dataroot and analysisroot folder names above!!")
print('dataroot =',repr(dataroot))
print('analysisroot =',repr(analysisroot))

In [ ]:
# List the samples available:
segmenter_gui.printsamples(dataroot)

In [ ]:
# USER: Decide which sample
sample = 'FeAu_0p5_tR'

In [ ]:
# List the datasets for that sample:
segmenter_gui.printdatasets( dataroot, sample )

In [ ]:
# USER: Decide which dataset
dataset = "ff1"

In [ ]:
# create ImageD11 dataset object

ds = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                        analysisroot=analysisroot,
                                        sample=sample,
                                        dset=dataset,
                                        detector=detector,
                                        omegamotor=omegamotor,
                                        dtymotor=dtymotor)
ds.import_all(scans=["1.1"])
ds.splinefile = splinefile
ds.maskfile = maskfile
ds.bgfile = bgfile
ds.save()

In [ ]:
ui = segmenter_gui.FrelonSegmenterGui(ds, worker, process, **options)

In [ ]:
options = ui.getopts()

In [ ]:
# now we run the segmenter on all our data

nthreads = len(os.sched_getaffinity(os.getpid()))

cf_2d, cf_3d = process(ds, nthreads-1, options)

In [ ]:
# we can use this to verify that the 3D merging is behaving as expected
# don't worry about this too much!

# take a few 3d peaks with the most 2d peaks, plot them

unique, counts = np.unique(cf_2d.spot3d_id, return_counts=True)
hits_dict = dict(zip(unique, counts))
hits_dict_max = sorted(hits_dict.items(), key=lambda x: x[1], reverse=True)

m = np.isin(cf_3d.index, [spot3d_id for spot3d_id, count in hits_dict_max[500:501]])
cf_3d_single_peak = cf_3d.copy()
cf_3d_single_peak.filter(m)

peak_2d_mask = np.isin(cf_2d.spot3d_id, cf_3d_single_peak.index)
cf_2d_peaks = cf_2d.copy()
cf_2d_peaks.filter(peak_2d_mask)

fig, ax = plt.subplots()
ax.scatter(cf_3d_single_peak.f_raw, cf_3d_single_peak.s_raw, marker="X", c=cf_3d_single_peak.omega, s=50, label='Merged 3D peak')
cols = ax.scatter(cf_2d_peaks.f_raw, cf_2d_peaks.s_raw, c=cf_2d_peaks.o_raw, s=cf_2d_peaks.s_I / 1000, label='Contibutory 2D peaks')
fig.colorbar(cols)
ax.set_xlim(0, 2048)
ax.set_ylim(0, 2048)
ax.invert_yaxis()
ax.legend()
ax.set_title("Color is omega of peak. Scaled by sum intensity")
ax.set_xlabel("f_raw")
ax.set_ylabel("s_raw")
plt.show()

In [ ]:
cf_2d = correct_cf_with_spline(cf_2d, splinefile)

In [ ]:
cf_3d = correct_cf_with_spline(cf_3d, splinefile)

In [ ]:
parfile = os.path.join(ds.analysisroot, 'Fe_tdxrd_refined.par')

In [ ]:
cf_2d.parameters.loadparameters(parfile)

cf_2d.updateGeometry()
ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)

In [ ]:
cf_3d.parameters.loadparameters(parfile)
cf_3d.updateGeometry()
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile)

In [ ]:
ds.parfile = parfile
ds.save()

In [ ]:
# change to 0 to allow all cells to be run automatically
if 1:
    raise ValueError("Hello!")

In [ ]:
# Now that weparfile happy with our indexing parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR": []
}

dset_prefix = "ff"

sample_list = ["FeAu_0p5_tR"]
    
samples_dict = utils.find_datasets_to_process(ds.dataroot, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_200um"]}

nthreads = len(os.sched_getaffinity(os.getpid()))

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        print("Importing DataSet object")
        ds = ImageD11.sinograms.dataset.DataSet(dataroot=ds.dataroot,
                                            analysisroot=ds.analysisroot,
                                            sample=sample,
                                            dset=dataset,
                                            detector=detector,
                                            omegamotor=omegamotor,
                                            dtymotor=dtymotor)
        
        if os.path.exists(ds.col2dfile):
            print(f"Found existing cf_2d for {dataset} in {sample}, skipping")
            continue
        
        ds.import_all(scans=["1.1"])
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        ds.save()
        
        ds.splinefile = splinefile
        ds.maskfile = maskfile
        ds.bgfile = bgfile

        print("Peaksearching")
        cf_2d, cf_3d = process(ds, nthreads-1, options)
        
        print("Spatially correcting peaks")
        cf_2d = correct_cf_with_spline(cf_2d, splinefile)
        cf_3d = correct_cf_with_spline(cf_3d, splinefile)
        
        print("Saving peaks to file")
        cf_2d.parameters.loadparameters(parfile)

        cf_2d.updateGeometry()
        ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)
        
        cf_3d.parameters.loadparameters(parfile)
        cf_3d.updateGeometry()
        ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile)
        
        ds.parfile = parfile
        ds.save()